<a href="https://colab.research.google.com/github/ksushrut/CoinClassifier/blob/main/Adv_CV_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coin Classification

## Guidelines
1) Please import packages in the import section only.

2) Use sensible variable names. If possible try to describe the use of the variables in the variable description section.

3) Github commits and branches?

4) set device to gpu. No cpu!


# PIP Install statements

In [4]:
!pip install colab_ssh --upgrade


# Import packages here

In [ ]:
# write only import statements here
import torch

# Variable Description:

##### Describe the essential variables here. preferable datatype and use. This is to make sure we understand the code we are using
eg: Xtrain: tensor, stores images for training